In [15]:
import numpy as np
import tensorflow as tf
import os, shutil
import pandas as pd

original_path = 'C:/Users/jyj98/Desktop/fer_images'
new_path = 'C:/Users/jyj98/Desktop/Dataset_for2'
example = pd.read_csv(r'C:\Users\jyj98\Desktop\data\icml_face_data.csv')
os.mkdir(new_path)

train_dir = os.path.join(new_path, 'train')
validation_dir = os.path.join(new_path, 'validation')

os.mkdir(train_dir)
os.mkdir(validation_dir)

In [16]:
train_angry = os.path.join(train_dir,'angry')
train_scared = os.path.join(train_dir,'scared')
train_happy = os.path.join(train_dir,'happy')
train_sad = os.path.join(train_dir,'sad')
train_neutral = os.path.join(train_dir,'neutral')
train_out = os.path.join(train_dir,'out')

os.mkdir(train_angry)
os.mkdir(train_scared)
os.mkdir(train_happy)
os.mkdir(train_sad)
os.mkdir(train_neutral)

validation_angry = os.path.join(validation_dir,'angry')
validation_scared = os.path.join(validation_dir,'scared')
validation_sad = os.path.join(validation_dir,'sad')
validation_happy = os.path.join(validation_dir,'happy')
validation_neutral = os.path.join(validation_dir,'neutral')
validation_out = os.path.join(validation_dir,'out')

os.mkdir(validation_angry)
os.mkdir(validation_scared)
os.mkdir(validation_happy)
os.mkdir(validation_sad)
os.mkdir(validation_neutral)



In [17]:
train_out = os.path.join(train_dir,'out')
validation_out = os.path.join(validation_dir,'out')
os.mkdir(train_out)
os.mkdir(validation_out)

In [18]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [22]:

fnames = ['{}.jpg'.format(i) for i in range(28709)]
i=0

for fname in fnames:
#     if angry
    src = os.path.join(original_path, fname) 
    if example.iloc[i].emotion == 0:
        
        dst = os.path.join(train_angry, fname)
#     if sad
    elif example.iloc[i].emotion == 4:
        
        dst = os.path.join(train_sad,fname)
#         if happy
    elif example.iloc[i].emotion == 3:
     
        dst = os.path.join(train_happy, fname)
#         if scared
    elif example.iloc[i].emotion == 2:
        
        dst = os.path.join(train_scared, fname)
    elif example.iloc[i].emotion == 6:
        
        dst = os.path.join(train_neutral, fname)
        
    else:
        dst = os.path.join(train_out,fname)
    shutil.copyfile(src, dst)
    i+=1


fnames = ['{}.jpg'.format(i) for i in range(28709, 35897)]
for fname in fnames:
    src = os.path.join(original_path, fname)
    if example.iloc[i].emotion == 0:
      
        dst = os.path.join(validation_angry, fname)
    elif example.iloc[i].emotion == 4:
        
        dst = os.path.join(validation_sad, fname)
    elif example.iloc[i].emotion == 3:
       
        dst = os.path.join(validation_happy, fname)
    elif example.iloc[i].emotion == 2:
      
        dst = os.path.join(validation_scared, fname)
    elif example.iloc[i].emotion == 6:

        dst=os.path.join(validation_neutral,fname)
    else:
        dst = os.path.join(validation_out,fname)
    shutil.copyfile(src, dst)
    i+=1


IndexError: single positional indexer is out-of-bounds

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./256,
                                  rotation_range=40,
                                  height_shift_range=0.3,
                                  width_shift_range=0.3,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./256)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    color_mode='grayscale',
    target_size=(48, 48)
    , batch_size=32
    , class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    color_mode='grayscale',
    target_size=(48, 48)
    , batch_size=32,
    class_mode='categorical'
)


Found 19840 images belonging to 5 classes.
Found 4640 images belonging to 5 classes.


In [30]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48,48,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))


model.compile(optimizer=optimizers.Adam(lr=0.001) ,loss='categorical_crossentropy', metrics=['acc'])


In [31]:
from PIL import Image
history = model.fit_generator(
    train_generator,
    steps_per_epoch=3745,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=145
)

Epoch 1/100
3745/3745 [==============================] - 116s 31ms/step - loss: 1.6020 - acc: 0.2294 - val_loss: 1.5804 - val_acc: 0.2776
Epoch 2/100
3745/3745 [==============================] - 113s 30ms/step - loss: 1.5865 - acc: 0.2594 - val_loss: 1.5595 - val_acc: 0.2881
Epoch 3/100
3745/3745 [==============================] - 119s 32ms/step - loss: 1.5663 - acc: 0.2850 - val_loss: 1.5137 - val_acc: 0.3287
Epoch 4/100
3745/3745 [==============================] - 113s 30ms/step - loss: 1.5402 - acc: 0.3098 - val_loss: 1.4514 - val_acc: 0.3690
Epoch 5/100
3745/3745 [==============================] - 113s 30ms/step - loss: 1.5112 - acc: 0.3313 - val_loss: 1.3914 - val_acc: 0.3940
Epoch 6/100
3745/3745 [==============================] - 113s 30ms/step - loss: 1.4767 - acc: 0.3528 - val_loss: 1.3379 - val_acc: 0.4345
Epoch 7/100
3745/3745 [==============================] - 111s 30ms/step - loss: 1.4482 - acc: 0.3712 - val_loss: 1.3193 - val_acc: 0.4478
Epoch 8/100
3745/3745 [===========

In [32]:
model.save('acc_0.54.h5')
print(validation_generator.class_indices)

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3, 'scared': 4}
